<a href="https://colab.research.google.com/github/pranjalsinha1965/AI-ML-CodingNinjas/blob/main/facial_expression_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214 (from 1)
Receiving objects: 100% (14214/14214), 239.65 MiB | 29.78 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Updating files: 100% (13996/13996), done.


In [ ]:
import csv
data = {}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader)
  for row in reader:
    key = row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key] = [row[1]]

In [ ]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [ ]:
import os

# Define the base directory
base_dir = "/content/master_data"

# Create directories safely
os.makedirs(os.path.join(base_dir, "training"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "testing"), exist_ok=True)

print("Directories created successfully!")


Directories created successfully!


In [ ]:
import os

# Define emotion list
emotion_list = ["anger", "happy", "sad", "neutral"]  # Add all required emotions

# Define base directories
base_dirs = ["master_data", "sample_data/emotion"]

# Create emotion folders inside each base directory
for base_dir in base_dirs:
    for emotion in emotion_list:
        os.makedirs(os.path.join(base_dir, emotion), exist_ok=True)

print("All directories are created successfully!")


All directories are created successfully!


In [ ]:
import os
from shutil import copyfile

split_size = 0.8

for emotion, images in data.items():
    train_size = int(split_size * len(images))
    train_images = images[:train_size]
    test_images = images[train_size:]

    # Ensure destination directories exist
    train_dest_dir = os.path.join('/content/master_data/training', emotion)
    test_dest_dir = os.path.join('/content/master_data/testing', emotion)

    os.makedirs(train_dest_dir, exist_ok=True)
    os.makedirs(test_dest_dir, exist_ok=True)

    for image in train_images:
        source = os.path.join('/content/facial_expressions/images', image)
        dest = os.path.join(train_dest_dir, image)

        if os.path.exists(source):  # Check if the source file exists
            copyfile(source, dest)
        else:
            print(f"Warning: Source file not found - {source}")

    for image in test_images:
        source = os.path.join('/content/facial_expressions/images', image)
        dest = os.path.join(test_dest_dir, image)

        if os.path.exists(source):  # Check if the source file exists
            copyfile(source, dest)
        else:
            print(f"Warning: Source file not found - {source}")


In [ ]:
!pip install --upgrade tensorflow


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
from tensorflow.keras import Input

model = tf.keras.models.Sequential([
    Input(shape=(100, 100, 3)),  # Explicit Input layer
    Conv2D(16, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(8, activation='softmax')
])

# Use the correct Adam optimizer syntax
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 98, 98, 16)          │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 49, 49, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 47, 47, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 21, 21, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 6400)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       6,554,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │           8,200 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,586,408 (25.13 MB)

 Trainable params: 6,586,408 (25.13 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
train_dir = '/content/master_data/training'
test_dir = '/content/master_data/testing'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
                                                    train_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                                                    test_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [ ]:
es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_acc', patience=3, mode='max', restore_best_weights=True)

model.fit(
    train_generator,
    epochs=10,
    verbose=1,
    validation_data=test_generator,
    callbacks=[es]
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 24s 219ms/step - accuracy: 0.4518 - loss: 2.8296 - val_accuracy: 0.4198 - val_loss: 1.0497
Epoch 2/10
 1/86 ━━━━━━━━━━━━━━━━━━━━ 13s 154ms/step - accuracy: 0.3281 - loss: 1.0148

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: accuracy,loss,val_accuracy,val_loss
  current = self.get_monitor_value(logs)


86/86 ━━━━━━━━━━━━━━━━━━━━ 15s 169ms/step - accuracy: 0.4997 - loss: 1.0336 - val_accuracy: 0.5718 - val_loss: 1.0604
Epoch 3/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 14s 167ms/step - accuracy: 0.6696 - loss: 0.8855 - val_accuracy: 0.6400 - val_loss: 0.9123
Epoch 4/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 14s 166ms/step - accuracy: 0.7413 - loss: 0.7435 - val_accuracy: 0.6240 - val_loss: 1.0141
Epoch 5/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 15s 172ms/step - accuracy: 0.7590 - loss: 0.6981 - val_accuracy: 0.6364 - val_loss: 1.0673
Epoch 6/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 15s 174ms/step - accuracy: 0.7732 - loss: 0.6562 - val_accuracy: 0.6433 - val_loss: 1.1962
Epoch 7/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 15s 169ms/step - accuracy: 0.7919 - loss: 0.6224 - val_accuracy: 0.6240 - val_loss: 1.3214
Epoch 8/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 15s 169ms/step - accuracy: 0.7850 - loss: 0.6214 - val_accuracy: 0.6477 - val_loss: 1.2719
Epoch 9/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 14s 167ms/step - accuracy: 0.8000 - loss: 0.5858 - val_accuracy: 0.649

# **APIs and its calls**

In [2]:
import requests

url = "https://codingninjas.in/api/v3/events"
resp = requests.get(url, allow_redirects=True)

print(f"Final URL: {resp.url}")  # Check where it's redirected
print(f"Status Code: {resp.status_code}")
print(f"Response Headers: {resp.headers}")

if "application/json" in resp.headers.get("Content-Type", ""):
    print(resp.json())  # Parse JSON if available
else:
    print("Response is not JSON. API might be restricted or the endpoint has changed.")


Final URL: https://www.naukri.com/code360/events
Status Code: 200
Response Headers: {'Content-Type': 'text/html', 'x-amz-id-2': 'ZEnYJ7wm8Ixe+Uv0eh/XOzgqd/ZEdhkufWgwK4bLV9PovGusAEdUom+IcHs9NUol+OUbeophkrG7TfSXkR6sx6e0LlhFEn20x0xF5KosCC0=', 'x-amz-request-id': '7DJH9V0DDDGSC1SP', 'Last-Modified': 'Fri, 28 Feb 2025 11:10:55 GMT', 'Server': 'AmazonS3', 'Content-Encoding': 'gzip', 'Cache-Control': 'public,max-age=300,must-revalidate', 'ETag': 'W/"afcbc497bdded3c352240afa2adb9b42"', 'X-Amz-Cf-Pop': 'BLR50-C2', 'X-Amz-Cf-Id': 'rmDIciMvTTjmHs-WBc7FpfYcJo9LwAmDERYGVlJAUKdG3YArX5mi1A==', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Length': '6133', 'X-EdgeConnect-MidMile-RTT': '212', 'X-EdgeConnect-Origin-MEX-Latency': '115', 'Date': 'Fri, 28 Feb 2025 19:02:42 GMT', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Set-Cookie': '_t_ds=cb318221740769361-7cb31822-0cb31822; expires=Sat, 28-Feb-2026 19:02:42 GMT; path=/; domain=.naukri.com; secure; HttpOnly; SameSite=Lax', 'Strict-Transport-Secu

In [19]:
## https://codingninjas.in/api/v3/events
## https://api.codingninjas.com/api/v3/courses
## https://api.github.com/events
## https://dog.ceo/api/breeds/list/all
import requests
response = requests.get('https://api.codingninjas.com/api/v3/courses')
print(response.json)
print(response.status_code)
print(response.encoding)
print(response.url)
print(response.text)
print(response.headers)
header_info = response.headers
print(header_info['Date'])
print(header_info['Content-type'])

<bound method Response.json of <Response [200]>>
200
utf-8
https://api.codingninjas.com/api/v3/courses
{"data":{"courses":[{"id":73,"title":"System Design Course","online_title":"system-design-courses","offline_title":"","name":"System Design Course","level":"Foundation Courses","brief_intro":"Brief Intro","preview_image_url":"https://ninjasfiles.s3.amazonaws.com/0000000000001178.png","available_online":true,"available_offline":false,"is_career_track":false,"course_group_id":1,"classroom_icon_url":"https://files.codingninjas.com/system-design-icon-11401.svg"},{"id":1,"title":"C++ \u0026 Data Structures","online_title":"onlline-c-plus-plus-course","offline_title":"classroom-c-plus-plus-course","name":"C++ Foundation with Data Structures","level":"Foundation Courses","brief_intro":"Brief Intro","preview_image_url":"https://files.codingninjas.in/0000000000001176.png","available_online":true,"available_offline":false,"is_career_track":false,"course_group_id":1,"classroom_icon_url":"https:/

In [24]:
import requests
response = requests.get("https://api.wheretheiss.at/v1/satellites/25544")
header = response.headers
print(header.get('Content-Type'))


application/json


In [50]:
import requests
import json
url = "https://api.codingninjas.com/api/v3/courses"
response = requests.get(url)
if response.status_code == 200:
        python_data = response.json()
        print(type(python_data))
        if 'data' in python_data and 'courses' in python_data['data']:
            all_courses = python_data['data']['courses']
            print(all_courses)


<class 'dict'>
[{'id': 73, 'title': 'System Design Course', 'online_title': 'system-design-courses', 'offline_title': '', 'name': 'System Design Course', 'level': 'Foundation Courses', 'brief_intro': 'Brief Intro', 'preview_image_url': 'https://ninjasfiles.s3.amazonaws.com/0000000000001178.png', 'available_online': True, 'available_offline': False, 'is_career_track': False, 'course_group_id': 1, 'classroom_icon_url': 'https://files.codingninjas.com/system-design-icon-11401.svg'}, {'id': 1, 'title': 'C++ & Data Structures', 'online_title': 'onlline-c-plus-plus-course', 'offline_title': 'classroom-c-plus-plus-course', 'name': 'C++ Foundation with Data Structures', 'level': 'Foundation Courses', 'brief_intro': 'Brief Intro', 'preview_image_url': 'https://files.codingninjas.in/0000000000001176.png', 'available_online': True, 'available_offline': False, 'is_career_track': False, 'course_group_id': 1, 'classroom_icon_url': 'https://files.codingninjas.in/0000000000003771.png'}, {'id': 45, 'ti

In [28]:
import json

json_data = '{"Student": "Mohit"}'  # JSON string (notice the double quotes around keys)
python_data = json.loads(json_data)

print(python_data)
print(type(python_data))  # Output: <class 'dict'>
print(python_data['Student'])

{'Student': 'Mohit'}
<class 'dict'>
Mohit


In [35]:
import json
json_data = '{"roll_number": null}'
python_data = json.loads(json_data)
print(python_data["roll_number"])
print(type(python_data["roll_number"]))

None
<class 'NoneType'>


In [42]:
import json
json_data = '{"student" : {"Name" : "Mohit", "Roll_No" : 101}}'
python_data = json.loads(json_data)
print(python_data)
student_details = python_data['student']
print(student_details)
print(student_details['Name'])
print(student_details['Roll_No'])

{'student': {'Name': 'Mohit', 'Roll_No': 101}}
{'Name': 'Mohit', 'Roll_No': 101}
Mohit
101


In [45]:
json_data = '{"Fruits" : ["Apple", "Banana", "Mango", "Cherry"]}'
python_data = json.loads(json_data)
print(python_data)
print(type(python_data))

{'Fruits': ['Apple', 'Banana', 'Mango', 'Cherry']}
<class 'dict'>


In [52]:
import json
python_data = json.loads(response.text)
print(type(python_data))
all_courses = python_data['data']['courses']
print(type(all_courses))

for i in all_courses:
  print(i['title'])

print(len(all_courses))

<class 'dict'>
<class 'list'>
System Design Course
C++ & Data Structures
Premium Android Course
Premium Machine Learning Course
Data structures and Algorithms [C++/Java/Python]
Front End (React)
Competitive Programmer
Premium Competitive Course
Premium Web Dev Course
Machine Learning
Premium Data Science Course
Android Developer
Web Dev (Full Stack)
Java & Data Structures
Data Science
Grokking the Operating Systems Interview
Python & Web Dev (MERN)
Competitive Programming
C++ & Web Dev (Node.js)
Android (Kotlin)
Data Scientist
Aptitude Preparation
Machine Learning
23


In [54]:
p = requests.get("https://dog.ceo/api/breeds/list/all")
print(p.text)
python_data = p.json()
print(type(python_data))
for key, val in python_data['message'].items():
  print(key, ':', len(val))

for i in python_data['message']['spaniel']:
  print(i)

{"message":{"affenpinscher":[],"african":[],"airedale":[],"akita":[],"appenzeller":[],"australian":["kelpie","shepherd"],"bakharwal":["indian"],"basenji":[],"beagle":[],"bluetick":[],"borzoi":[],"bouvier":[],"boxer":[],"brabancon":[],"briard":[],"buhund":["norwegian"],"bulldog":["boston","english","french"],"bullterrier":["staffordshire"],"cattledog":["australian"],"cavapoo":[],"chihuahua":[],"chippiparai":["indian"],"chow":[],"clumber":[],"cockapoo":[],"collie":["border"],"coonhound":[],"corgi":["cardigan"],"cotondetulear":[],"dachshund":[],"dalmatian":[],"dane":["great"],"danish":["swedish"],"deerhound":["scottish"],"dhole":[],"dingo":[],"doberman":[],"elkhound":["norwegian"],"entlebucher":[],"eskimo":[],"finnish":["lapphund"],"frise":["bichon"],"gaddi":["indian"],"germanshepherd":[],"greyhound":["indian","italian"],"groenendael":[],"havanese":[],"hound":["afghan","basset","blood","english","ibizan","plott","walker"],"husky":[],"keeshond":[],"kelpie":[],"kombai":[],"komondor":[],"kuv

**Next round of questions and all**

In [58]:
import requests

url = "https://dog.ceo/api/breeds/list/all"
response = requests.get(url)
py_data = response.json()
if 'message' in py_data and 'hound' in py_data['message']:
    for i in py_data['message']['hound']:
        print(i)

afghan
basset
blood
english
ibizan
plott
walker


In [59]:
import requests as rqt
response = rqt.get("https://dog.ceo/api/breeds/list/all")
py_data = response.json()
print(len(py_data['message']['poodle']))

4


In [61]:
import requests
response = requests.get("https://api.codingninjas.in/api/v3/courses")
py_data = response.json()
for i in py_data['data']['courses']:
  if(i['available_online'] == True):
    print(i['name'].strip())

System Design Course
C++ Foundation with Data Structures
Premium Android App Development
Premium Machine Learning Course
Premium Data structures and Algorithms [C++/Java/Python]
Advanced  Front-End Web Development with React
Ninja Competitive Programmer Track
Premium Competitive Programming Course
Premium Full Stack Web Development
Machine Learning & Deep Learning Course
Premium Data Science Course
Ninja Android Developer Career Track
Full Stack Web Development
JAVA Foundation with Data Structures
Data Science & Machine Learning Complete
Grokking the Operating Systems Interview
Ninja  Web Developer Career Track - NodeJS & ReactJs
Competitive Programming Course
Ninja  Web Developer Career Track - NodeJS
Android Development with Kotlin
Ninja Data Scientist Career Track
Aptitude Preparation Course
Ninja Machine Learning Engineer Career Track


In [65]:
a = requests.get("https://api.openaq.org/v1/cities", params = {'country' : 'AE', 'limit' : 1})
print(a.status_code)
print(a.json())
print(a.url)

410
{'message': 'Gone. Version 1 and Version 2 API endpoints are retired and no longer available. Please migrate to Version 3 endpoints.'}
https://api.openaq.org/v1/cities?country=AE&limit=1


In [70]:
# Title of the course
import requests
url = "https://swapi.dev/api/planets/"
response = requests.get(url, params={"page": 2})
data = response.json()
if 'results' in data:
    for planet in data['results']:
        print(planet['name'])

Geonosis
Utapau
Mustafar
Kashyyyk
Polis Massa
Mygeeto
Felucia
Cato Neimoidia
Saleucami
Stewjon


In [4]:
# Total number of cities
import requests
url = "https://swapi.dev/api/planets/"
response = requests.get(url, params={"search": "and"})
data = response.json()
if 'results' in data:
  for planet in data['results']:
    print(planet['name'])

Trandosha
Chandrila


In [11]:
import requests as rqt
resp = rqt.get("https://api.openaq.org/v1/cities", params = {"country" : "BE", "limit" : 5, "order_by" : "city"})
data = resp.json()
if "results" in data:
  for i in data["results"]:
    print(i["city"])